In [5]:
import torch.nn as nn
import torch
from contextlib import nullcontext
import time
from tqdm.notebook import tqdm
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

# 说明：
本次训练完全自己造轮子
不使用transformer的trainer

# 建立整体框架
我们先不写具体代码，先把大框架设计好，再往里面填东西
*配置与参数*：
- 神经

# 配置与参数解析
我们首先要设计好，训练的一些参数

In [4]:

from model.model_minimind import MiniMindConfig
class TrainArgs():
    #文件管理
    out_dir = './pretrain_output'
    checkpoint_path = "./pretrain_output/latest_checkpoint.pth"
    data_path = '../data/pretrain_hq.jsonl'
    #神经网络训练管理
    epochs = 2
    batch_size = 16
    accumulation_steps = 4
    learning_rate = 5e-4
    warm_up = 0
    grad_clip = 1
    dtype = 'bfloat16'
    device = "cuda:0" if torch.cuda.is_available() else "cpu"
    num_workers = 0
    log_interval = 50
    save_interval = 2000
    ctx = nullcontext() if device == "cpu" else torch.cuda.amp.autocast()
class LLMargs():
    use_moe = True
    hidden_size = 512
    num_hidden_layers = 8
    

lm_config = MiniMindConfig(use_moe=LLMargs.use_moe,hidden_size=LLMargs.hidden_size,num_hidden_layers=LLMargs.num_hidden_layers)


NameError: name 'torch' is not defined

In [8]:
os.makedirs(TrainArgs.save_dir, exist_ok=True)
os.makedirs(TrainArgs.out_dir, exist_ok=True)

## 环境与设备初始化

In [42]:
import torch

print(f"CUDA is available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"Number of GPUs: {torch.cuda.device_count()}")
    for i in range(torch.cuda.device_count()):
        print(f"  GPU {i}: {torch.cuda.get_device_name(i)}")
        print(f"    Memory: {torch.cuda.get_device_properties(i).total_memory / (1024**3):.2f} GB")

CUDA is available: True
Number of GPUs: 1
  GPU 0: NVIDIA GeForce RTX 4060 Laptop GPU
    Memory: 8.00 GB


# 模型加载、数据加载

In [3]:
from transformers import AutoTokenizer
from model.model_minimind import MiniMindConfig, MiniMindForCausalLM
tokenizer = AutoTokenizer.from_pretrained('../model/')
model = MiniMindForCausalLM(lm_config).to(TrainArgs.device)

NameError: name 'lm_config' is not defined

In [4]:
print(model)

total_params = sum(p.numel() for p in model.parameters())

# 打印结果，使用 f-string 的格式化功能让数字更易读（例如，加上千位分隔符）
print(f"模型总参数量 (Total Parameters): {total_params:,}")

# 如果你还想区分“可训练参数”，可以这样做：
# 在大多数情况下，总参数量和可训练参数量是一样的。
# 除非你手动设置了某些参数的 requires_grad=False (冻结了某些层)
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"可训练参数量 (Trainable Parameters): {trainable_params:,}")

MiniMindForCausalLM(
  (model): MiniMindModel(
    (embed_tokens): Embedding(6400, 512)
    (dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-7): 8 x MiniMindBlock(
        (self_attn): Attention(
          (q_proj): Linear(in_features=512, out_features=512, bias=False)
          (k_proj): Linear(in_features=512, out_features=128, bias=False)
          (v_proj): Linear(in_features=512, out_features=128, bias=False)
          (o_proj): Linear(in_features=512, out_features=512, bias=False)
          (attn_dropout): Dropout(p=0.0, inplace=False)
          (resid_dropout): Dropout(p=0.0, inplace=False)
        )
        (input_layernorm): RMSNorm()
        (post_attention_layernorm): RMSNorm()
        (mlp): MOEFeedForward(
          (experts): ModuleList(
            (0-3): 4 x FeedForward(
              (gate_proj): Linear(in_features=512, out_features=1408, bias=False)
              (down_proj): Linear(in_features=1408, out_features=512, bias=False)
           

对于 Dataset 类，它通常要求其子类至少实现两个特殊方法：

`__len__(self)`: 这个方法应该返回数据集中样本的总数量。PyTorch 的 DataLoader 会调用它来知道数据集的大小。

`__getitem__(self, index)`: 这个方法是核心，它应该根据给定的 index（索引）返回数据集中的一个样本。DataLoader 会通过这个方法来逐个或批量获取数据

In [5]:
#定义处理数据的类
from torch.utils.data import Dataset, DataLoader, Subset
import json
class PretrainDataset(Dataset):
    def __init__(self,tokenizer,data_path,max_length=512):
        super().__init__()
        self.tokenizer = tokenizer
        self.samples = self.load_data(data_path)
        self.max_length = max_length

    def load_data(self,data_path):
        samples = []
        with open(data_path,'r',encoding='utf-8') as f:
            for line in f:
                data = json.loads(line.strip())
                samples.append(data)
        return samples
    
    def __len__(self):
        return len(self.samples)
    
    def __getitem__(self, index) :
        sample = self.samples[index]

        encoding = self.tokenizer(
            sample['text'],
            max_length = self.max_length,
            padding = 'max_length',
            truncation = True,
            return_tensors='pt'
        )

        input_ids = encoding.input_ids.squeeze()
        loss_mask = (input_ids != self.tokenizer.pad_token_id)

        X = input_ids[:-1].clone().detach()
        Y = input_ids[1:].clone().detach()
        loss_mask = loss_mask[1:].clone().detach()

        return X,Y,loss_mask
        

In [6]:
train_ds = PretrainDataset(tokenizer,TrainArgs.data_path,max_length=512)


In [7]:
print(f'{train_ds[1]}\n长度为{len(train_ds)}')

(tensor([   1, 1227, 2346, 5914,  270, 6095,  801, 1415, 1232, 5354,  286,  201,
         434, 1157, 2042, 2504, 1422, 3458,  854,  888, 2537, 5371,  801, 1415,
        1232, 5354,   28,  223, 2037,  595,   17,  854,  888, 2537, 3458, 5923,
           2,  223,    1, 1086,  397, 5885, 2336, 3694,  269, 3858, 1994,  286,
        6243,  270, 5213, 1086,  803, 5885,  682, 2711, 3858, 1994,  286, 4600,
         803,  590, 1698, 3445, 2404,  371, 1971,  269, 3858, 1994, 3274,  814,
           2,  223,    1, 1086,  397, 2910,  541, 5804,  953,  854, 5221,  242,
        1531,  286, 1188,  270,  397, 1086,  803, 2910,  541, 2345, 1439, 1330,
         237, 5221,  242, 1531,  286, 4587, 1439, 1330,  237, 3993,  568, 1687,
         814, 1775, 4874, 1170,  345, 1476,  953,  446, 2207,    2,  223,    1,
        2525, 4584, 2706,  333, 1581, 2345,  611, 1781,  269, 3044, 1486, 3411,
         286, 1944,  103, 3344, 3139,  876,  118, 1749, 1729,  954,  270, 1781,
         954,  845, 1468, 1319,  954,  

加载dataloader

如果想检验整个训练流程，采用小数据检验是最合适的


In [26]:
test_ds = Subset(train_ds, range(500))
print(f'\n长度为{len(test_ds)}')


长度为500


In [27]:
test_loader = DataLoader(
    test_ds,
    batch_size = 4,
    shuffle = True,
    num_workers=1,
    pin_memory=True
)

In [33]:
train_loader = DataLoader(
    train_ds,
    batch_size = TrainArgs.batch_size,
    shuffle = True,
    num_workers = TrainArgs.num_workers,
    pin_memory= True
)

In [34]:
tag = False
for batch_idx, (X_batch, Y_batch, loss_mask_batch) in enumerate(train_loader):
    if not tag:
        print(f"Batch {batch_idx}:")
        print(f"  X_batch shape: {X_batch.shape}")
        print(f"  Y_batch shape: {Y_batch.shape}")
        print(f"  loss_mask_batch shape: {loss_mask_batch.shape}")
        tag = True
    break

Batch 0:
  X_batch shape: torch.Size([8, 511])
  Y_batch shape: torch.Size([8, 511])
  loss_mask_batch shape: torch.Size([8, 511])


In [28]:
tag = False
for batch_idx, (X_batch, Y_batch, loss_mask_batch) in enumerate(test_loader):
    if not tag:
        print(f"Batch {batch_idx}:")
        print(f"  X_batch shape: {X_batch.shape}")
        print(f"  Y_batch shape: {Y_batch.shape}")
        print(f"  loss_mask_batch shape: {loss_mask_batch.shape}")
        tag = True
    break

Batch 0:
  X_batch shape: torch.Size([4, 511])
  Y_batch shape: torch.Size([4, 511])
  loss_mask_batch shape: torch.Size([4, 511])


## 训练核心五步法
- 数据上设备
- 前向传播
- 计算损失
- 反向传播
- 更新权重

In [14]:
import math
from torch import optim
def get_lr(current_step, total_steps, lr):
    return lr / 10 + 0.5 * lr * (1 + math.cos(math.pi * current_step / total_steps))
optimizer = optim.AdamW(model.parameters(), lr=TrainArgs.learning_rate)
#据模型在训练数据上的表现来调整模型的参数  训练的一个大脑
scaler = torch.cuda.amp.GradScaler(enabled=(TrainArgs.dtype in ['float16', 'bfloat16']))
#自动混合精度训练工具 训练的一个助手

In [16]:
import swanlab
# 初始化swanlab，传入项目名、实验名等
swanlab.init(
    project="MiniMind-Pretrain", 
    experiment_name="MiniMind-Pretrain", 
    config=vars(TrainArgs()) # 将你的所有超参数配置一次性传给swanlab
)

swanlab: Tracking run with swanlab version 0.6.7                                   
swanlab: Run data will be saved locally in /home/kkkkkk/ai_project/minimind/从0到1大模型全流程梳理/swanlog/run-20250719_204458-ebf8235a
swanlab: 👋 Hi kkaiaiai, welcome to swanlab!
swanlab: Syncing run MiniMind-Pretrain to the cloud
swanlab: 🏠 View project at https://swanlab.cn/@kkaiaiai/MiniMind-Pretrain
swanlab: 🚀 View run at https://swanlab.cn/@kkaiaiai/MiniMind-Pretrain/runs/xtpiiakzeio6vgsef1zs6


In [41]:

def train_epoch(epoch, model, train_loader, optimizer, scaler, lm_config):
    #选择初始化损失函数！
    model.train() # 确保模型处于训练模式

    loss_fct = nn.CrossEntropyLoss(reduction='none')

    start_time = time.time()#一会记录日志要用

    iter_per_epoch = len(train_loader)#看一下一次数据有多长，计算学习率要用

    # --- 新增：使用tqdm包装DataLoader以显示进度条 ---
    progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{TrainArgs.epochs}", leave=True)

    for step, (X, Y, loss_mask) in enumerate(train_loader):#不断提取数据
        X = X.to(TrainArgs.device)
        Y = Y.to(TrainArgs.device)
        loss_mask = loss_mask.to(TrainArgs.device)#数据上设备，上到gpu上

        
        lr = get_lr(epoch * iter_per_epoch + step, TrainArgs.epochs * iter_per_epoch, TrainArgs.learning_rate)
        for param_group in optimizer.param_groups:
            param_group['lr'] = lr                #计算当前学习率的算法

        with TrainArgs.ctx:
            res = model(X)#得到的是logits，aux_loss

            loss = loss_fct(
                res.logits.view(-1, res.logits.size(-1)),
                Y.view(-1)
            ).view(Y.size())  #计算loss的算法
            loss = (loss * loss_mask).sum() / loss_mask.sum()
            loss += res.aux_loss
            loss = loss / TrainArgs.accumulation_steps  

            scaler.scale(loss).backward()  #反向传播

        if (step + 1) % TrainArgs.accumulation_steps == 0:#判断梯度累加到位没有
            scaler.unscale_(optimizer)#*前置操作
            torch.nn.utils.clip_grad_norm_(model.parameters(), TrainArgs.grad_clip)
            #梯度裁剪
            scaler.step(optimizer)#参数更新
            scaler.update()#*调整

            optimizer.zero_grad(set_to_none=True)#本次训练结束，清零梯度，为下一次反向传播准备

        if step % TrainArgs.log_interval == 0:
            spend_time = time.time() - start_time
            current_lr = optimizer.param_groups[0]['lr']
            progress_bar.set_postfix({
                "loss": f"{loss.item():.3f}",
                "lr": f"{current_lr:.2e}"
            })

            if (swanlab is not None) : #用日志软件记录训练过程
                swanlab.log({"loss": loss.item() * TrainArgs.accumulation_steps,
                           "lr": optimizer.param_groups[-1]['lr'],
                           "epoch_Time": spend_time / (step + 1) * iter_per_epoch // 60 - spend_time // 60})

            # 保存checkpoint
        if (step + 1) % TrainArgs.save_interval == 0:
            # 只在更新步之后保存，确保梯度累积完成
            if (step + 1) % TrainArgs.accumulation_steps == 0:
                model.eval()
                print(f"\nSaving checkpoint at step {step+1}...")
                
                # 创建checkpoint字典
                checkpoint = {
                     # 对模型权重进行半精度转换
                    'model': {k: v.half() for k, v in model.state_dict().items()},
                    # 对优化器状态也进行转换（更严谨的做法）
                    'optimizer': {
                        'state': {k: v.half() if torch.is_tensor(v) else v for k, v in optimizer.state_dict()['state'].items()},
                        'param_groups': optimizer.state_dict()['param_groups']
                    },
                    'scaler': scaler.state_dict(),
                    'epoch': epoch,
                    'step': step,
                    'config': TrainArgs,
                }
                
                # 保存
                torch.save(checkpoint, TrainArgs.checkpoint_path)
                print("Checkpoint saved.")
                model.train() # 切换回训练模式



## 实现循环训练

正式训练前假如了这么一大堆逻辑，是为了这么个情节：
训练了一小时，有事，还必须关电脑，
那我们下次训练时就可以接着之前的chekpoint
接着之前的模型权重、模型eopoch

In [47]:
 # --- 关键：Checkpoint加载逻辑 ---
start_epoch = 0
start_step = 0
if os.path.exists(TrainArgs.checkpoint_path):
    print(f"=> Resuming from checkpoint: {TrainArgs.checkpoint_path}")
    checkpoint = torch.load(TrainArgs.checkpoint_path, map_location=TrainArgs.device)
    
    # 加载模型权重 (处理DDP保存的权重)
    model_state_dict = checkpoint['model']
    if any(key.startswith('module.') for key in model_state_dict):
            model_state_dict = {k.replace('module.', ''): v for k, v in model_state_dict.items()}
    model.load_state_dict(model_state_dict)
    
    # 加载优化器和scaler状态
    optimizer.load_state_dict(checkpoint['optimizer'])
    scaler.load_state_dict(checkpoint['scaler'])
    
    # 恢复训练进度
    start_epoch = checkpoint['epoch']
    # 注意：恢复step逻辑可以让dataloader跳过已训练数据，这里简化为从头开始当前epoch
    print(f"=> Resumed from epoch {start_epoch + 1}")
else:
    print("=> Starting from scratch...")

model.to(TrainArgs.device)

=> Resuming from checkpoint: ./output/latest_checkpoint.pth
=> Resumed from epoch 1


MiniMindForCausalLM(
  (model): MiniMindModel(
    (embed_tokens): Embedding(6400, 512)
    (dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-7): 8 x MiniMindBlock(
        (self_attn): Attention(
          (q_proj): Linear(in_features=512, out_features=512, bias=False)
          (k_proj): Linear(in_features=512, out_features=128, bias=False)
          (v_proj): Linear(in_features=512, out_features=128, bias=False)
          (o_proj): Linear(in_features=512, out_features=512, bias=False)
          (attn_dropout): Dropout(p=0.0, inplace=False)
          (resid_dropout): Dropout(p=0.0, inplace=False)
        )
        (input_layernorm): RMSNorm()
        (post_attention_layernorm): RMSNorm()
        (mlp): MOEFeedForward(
          (experts): ModuleList(
            (0-3): 4 x FeedForward(
              (gate_proj): Linear(in_features=512, out_features=1408, bias=False)
              (down_proj): Linear(in_features=1408, out_features=512, bias=False)
           

训练！ 外层epoch循环

In [ ]:
#测试训练运行以下代码
for epoch in range(start_epoch, TrainArgs.epochs):
        train_epoch(epoch, model, test_loader, optimizer, scaler, lm_config)

In [48]:
#正式训练运行以下代码
for epoch in range(start_epoch, TrainArgs.epochs):
        train_epoch(epoch, model, train_loader, optimizer, scaler, lm_config)

Epoch 1/2:   0%|          | 0/176638 [00:00<?, ?it/s]

KeyboardInterrupt: 